<a href="https://colab.research.google.com/github/alanpirotta/freecodecamp_certif/blob/main/fcc_predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you will predict healthcare costs using a regression algorithm.

You are given a dataset that contains information about different people including their healthcare costs. Use the data to predict healthcare costs based on new data.

The first two cells of this notebook import libraries and the data.

Make sure to convert categorical data to numbers. Use 80% of the data as the `train_dataset` and 20% of the data as the `test_dataset`.

`pop` off the "expenses" column from these datasets to create new datasets called `train_labels` and `test_labels`. Use these labels when training your model.

Create a model and train it with the `train_dataset`. Run the final cell in this notebook to check your model. The final cell will use the unseen `test_dataset` to check how well the model generalizes.

To pass the challenge, `model.evaluate` must return a Mean Absolute Error of under 3500. This means it predicts health care costs correctly within $3500.

The final cell will also predict expenses using the `test_dataset` and graph the results.

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
pip install tensorflow_decision_forests

In [ ]:
pip install wurlitzer

In [ ]:
import tensorflow_decision_forests as tfdf

try:
  from wurlitzer import sys_pipes
except:
  from colabtools.googlelog import CaptureLog as sys_pipes

In [ ]:
# Check the version of TensorFlow Decision Forests
print("Found TensorFlow Decision Forests v" + tfdf.__version__)

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

### Change categorical data to numbers.
Binary if possible

In [ ]:
feature = "region"
classes = dataset[feature].unique().tolist()
print(f"Feature classes: {classes}")

dataset[feature] = dataset[feature].map(classes.index)

# More manual Alternative 
# dataset['region'] = dataset['region'].replace({'southeast': 0,
#                                               'southwest': 1,
#                                               'northwest': 2,
#                                               'northeast': 3
#                                             })
print(dataset['region'].value_counts())
dataset.tail()

In [ ]:
dataset['smoker'] = np.where(dataset['smoker']=='yes',1,0)
print(dataset['smoker'].value_counts())
dataset.tail()

In [ ]:
dataset['sex'] = np.where(dataset['sex']=='male',1,0)
print(dataset['sex'].value_counts())
dataset.tail()

### Split between train and test datasets

In [ ]:
from sklearn.model_selection import train_test_split
train_dataset_pd, test_dataset_pd = train_test_split(dataset, test_size=0.2)
print(f'train shape: {train_dataset_pd.shape}')
print(f'test shape: {test_dataset_pd.shape}')
train_dataset_pd.tail()

In [ ]:
Q1 = train_dataset_pd['bmi'].quantile(0.25)
Q3 = train_dataset_pd['bmi'].quantile(0.75)  
IQR = Q3 - Q1
print (Q1 - 1.5 * IQR)
print (Q3 + 1.5 * IQR)
outliers = train_dataset_pd[(train_dataset_pd['bmi'] < (Q1 - 1.5 * IQR)) | (train_dataset_pd['bmi'] > (Q3 + 1.5 * IQR))]
train_dataset_pd = train_dataset_pd.drop(outliers.index)

In [ ]:
label = 'expenses'

train_dataset = tfdf.keras.pd_dataframe_to_tf_dataset(train_dataset_pd, label=label, task=tfdf.keras.Task.REGRESSION)
test_dataset = tfdf.keras.pd_dataframe_to_tf_dataset(test_dataset_pd, label=label, task=tfdf.keras.Task.REGRESSION)

In [ ]:
model = tfdf.keras.RandomForestModel(task = tfdf.keras.Task.REGRESSION)
model.compile(metrics=[["mae","mse"]])
with sys_pipes():
  model.fit(x=train_dataset)

In [ ]:
evaluation = model.evaluate(test_dataset, return_dict=True)
print(evaluation)

In [ ]:
test_labels = test_dataset_pd['expenses']
test_labels

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
